The Project is going to predict the next word like When we search on we will be getting some suggestios
in this projects stepts like
1.   Data Collection
2.   EDA
1.   Preprocessing
1.   Vectorization
1.   Padding
2.   LSTM
2.   Inference


In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding,LSTM,Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
import pickle
import numpy as np
import os


In [ ]:
files = open("/content/NLP_Data1.txt",'r',encoding="utf8")

lines = []

for i in files:
  lines.append(i)

In [ ]:

lines[1]

In [ ]:
data = ""
for i in lines:
  data = " ".join(lines)
data = data.replace('\n','').replace('\r','').replace('\ufeff','')

In [ ]:
data[:400]

In [ ]:
# Removing the Punctauation
import string
translator = str.maketrans(string.punctuation,' '*len(string.punctuation))
new_data = data.translate(translator)
new_data[:500]

In [ ]:
new = []
for i in data.split():
  if i not in new:
    new.append(i)
data = " ".join(new)

In [ ]:
data[:400]

In [ ]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts([data])

In [ ]:
pickle.dump(tokenizer,open("tokenizer.pkl","wb"))

In [ ]:
sequence_data = tokenizer.texts_to_sequences([data])[0]

In [ ]:
sequence_data[:10]

In [ ]:
vacab_size = len(tokenizer.word_index)+1

In [ ]:
vacab_size

In [ ]:
sequences = []
for i in range(1,len(sequence_data)):
  words = sequence_data[i-1:i+1]
  sequences.append(words)

In [ ]:
len(sequences)

In [ ]:
sequences = np.array(sequences)
sequences[:120]

In [ ]:
X = []
Y = []
for i in sequences:
  X.append(i[0])
  Y.append(i[0])
X = np.array(X)
Y = np.array(Y)


In [ ]:
X

In [ ]:
Y

In [ ]:
y = to_categorical(Y,num_classes=vacab_size)

In [ ]:
y[:10]

In [ ]:
model =  Sequential()

In [ ]:
model.add(Embedding(vacab_size, 10 ,input_length = 1))
model.add(LSTM(units = 1024,return_sequences = True))
model.add(LSTM(units = 1024,return_sequences = True))
model.add(LSTM(units = 1024))
model.add(Dense(1024,activation ="relu"))
model.add(Dense(vacab_size, activation = "softmax"))

In [ ]:
model.summary()

In [ ]:

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau,TensorBoard

checkpoint = ModelCheckpoint('model_nwp.h5',monitor = 'loss',verbose = 1,save_best_only = True, mode = 'auto')
reduces = ReduceLROnPlateau(monitor = 'loss',factor = 0.2,patience = 3,min_lr = 0.0001,verbose = 1)
logdir = 'logsnw'
tensorboard_vis = TensorBoard(log_dir = logdir)

In [ ]:

from tensorflow.keras.optimizers import Adam
model.compile(optimizer = Adam(lr = 0.001),loss = "categorical_crossentropy")


In [ ]:
model.fit(X,y, epochs = 75,batch_size=64,callbacks = [checkpoint,reduces,tensorboard_vis])


In [ ]:
%load_ext tensorboard
%tensorboard --logdir="./logsnw"

In [ ]:
def predictive_text_next(text):
  sequence = tokenizer.texts_to_sequences([text])[0]
  sequence = np.array(sequence)
  preds = model.predict(sequence)
  p_class = np.argmax(preds)
  p_word = tokenizer.index_word[p_class]

  return p_word

In [ ]:
text = input("Enter Text : ")
text = text.split(' ')
text = text[-1]
text = ''.join(text)
predictive_text_next(text)